In [ ]:
# import environment
from marketsai.mon_policy.env_mon_infin_final import MonPolicy
# import scipy.io as sio
# from scipy.interpolate import RegularGridInterpolator
from scipy.stats import linregress
from marketsai.utils import encode
import matplotlib.pyplot as plt
import pandas as pd
import os
import random


# from sklearn import linear_model
import numpy as np
import seaborn as sn
import math
import csv
import json
from ray.rllib.agents.ppo import PPOTrainer
from ray.tune.registry import register_env
from ray import shutdown, init

""" GLOBAL CONFIGS """
# Script Options
NATIVE = False
TEST = True  # for publication
SAVE_CSV = False  # save learning CSV
PLOT_HIST = True
PLOT_PROGRESS = False  # create plot with progress
SIMUL_EPISODES = 1
ENV_HORIZON = 60
EVAL_RESULTS = True
CHKPT_SELECT_REF = True
RESULTS_REF = np.array([1.3, 0.12, 0.1, 0.0005])
CHKPT_SELECT_MANUAL = False
CHKPT_id = 0
CHKPT_SELECT_MIN = False
CHKPT_SELECT_MAX = False
BETA = 0.95 ** (1 / 12)

# register environment
env_label = "mon_policy"
register_env(env_label, MonPolicy)

# Output Directories

if TEST:
    if NATIVE:
        OUTPUT_PATH_EXPERS = (
            "/Users/matiascovarrubias/Dropbox/RL_macro/Experiments/ALL/"
        )
        OUTPUT_PATH_FIGURES = (
            "/Users/matiascovarrubias/Dropbox/RL_macro/Documents/Figures/ALL/"
        )
        OUTPUT_PATH_RESULTS = "~/ray_results/ALL/"
    else:
        OUTPUT_PATH_EXPERS = "/scratch/mc5851/Experiments/ALL/"
        OUTPUT_PATH_FIGURES = "/scratch/mc5851/Figures/ALL/"
        OUTPUT_PATH_RESULTS = "/scratch/mc5851/ray_results/ALL/"

else:
    if NATIVE:
        OUTPUT_PATH_EXPERS = "/Users/matiascovarrubias/Dropbox/RL_macro/Experiments/"
        OUTPUT_PATH_FIGURES = (
            "/Users/matiascovarrubias/Dropbox/RL_macro/Documents/Figures/"
        )
        OUTPUT_PATH_RESULTS = "~/ray_results"
    else:
        OUTPUT_PATH_EXPERS = "/scratch/mc5851/Experiments/"
        OUTPUT_PATH_FIGURES = "/scratch/mc5851/Figures/"
        OUTPUT_PATH_RESULTS = "/scratch/mc5851/ray_results/"
# Plot options
sn.color_palette("Set2")
sn.set_style("ticks")  # grid styling, "dark"
# plt.figure(figure=(8, 4))
# choose between "paper", "talk" or "poster"
sn.set_context(
    "paper",
    font_scale=1.4,
)

# """ Step 0: import experiment data and initalize empty output data """
# with open(INPUT_PATH_EXPERS) as f:
#     exp_data_dict = json.load(f)


# # UNPACK USEFUL DATA
# num_trials = len(exp_data_dict["results_eval"][0][0])
# print(num_trials, exp_data_dict["results"])
# exp_names = exp_data_dict["exp_names"][0]
# checkpoints = exp_data_dict["checkpoints"][0]

# INPUT_PATH_CHECKPOINT = checkpoints[2]

# # progress_csv_dirs = exp_data_dict["progress_csv_dirs"]


# # best_rewards = exp_data_dict["best_rewards"]

# # useful functions
# def process_rewards(r, BETA):
#     discounted_r = np.zeros_like(r)
#     running_add = 0
#     for t in reversed(range(0, len(r))):
#         running_add = running_add * BETA + r[t]
#         discounted_r[t] = running_add
#     return discounted_r[0]


""" Step 2.0: replicate original environemnt and config """
# environment config
env_config = {
    "horizon": ENV_HORIZON,
    "n_inds": 200,
    "n_firms": 2,
    "eval_mode": False,
    "random_eval": False,
    "analysis_mode": False,
    "noagg": False,
    "regime_change": False,
    "infl_regime": "low",
}

env_config_eval = env_config.copy()
env_config_eval["eval_mode"] = True
env_config_analysis = env_config.copy()
env_config_analysis["analysis_mode"] = True
env_config_deviation = env_config_eval.copy()
env_config_deviation["deviation_mode"] = True
env_config_noagg = env_config_eval.copy()
env_config_noagg["no_agg"] = True
env_config_analysis_noagg = env_config_analysis.copy()
env_config_analysis_noagg["no_agg"] = True

# We instantiate the environment to extract information.
env = MonPolicy(env_config_eval)
config_algo = {
    "gamma": BETA,
    "env": env_label,
    "env_config": env_config_eval,
    "horizon": ENV_HORIZON,
    "explore": False,
    "framework": "torch",
    "multiagent": {
        "policies": {
            "firm_even": (
                None,
                env.observation_space[0],
                env.action_space[0],
                {},
            ),
            "firm_odd": (
                None,
                env.observation_space[0],
                env.action_space[0],
                {},
            ),
        },
        "policy_mapping_fn": (
            lambda agent_id: "firm_even" if agent_id % 2 == 0 else "firm_odd"
        ),
    },
}


In [ ]:
# Input Directories (of json file with experiment data)
INPUT_PATH_EXPERS_1 = "/scratch/mc5851/ray_results/server_mon_infin_1_exp_0_Nov15_PPO_run/"
TRIAL_PATH_1 = "PPO_mon_infin_1_5edcc_00000_0_2021-11-16_10-53-41/"
df_1_trial =pd.read_csv(INPUT_PATH_EXPERS_1+TRIAL_PATH_1+'progress.csv')


In [ ]:

results = {
    "Markups": [],
    "Freq. of Adj.": [],
    "Size of Adj.": [],
    "S.D. of log C": [],
    "Profits": [],
}

results_raw=[]

for i in range(34):

    if i<2:
        TRIAL_PATH_1 = f"/scratch/mc5851/ray_results/server_mon_infin_1_exp_0_Nov15_PPO_run/PPO_mon_infin_1_5edcc_0000{i}_{i}_2021-11-16_10-53-41/"
    elif i<10:
        TRIAL_PATH_1 = f"/scratch/mc5851/ray_results/server_mon_infin_1_exp_0_Nov15_PPO_run/PPO_mon_infin_1_5edcc_0000{i}_{i}_2021-11-16_10-53-42/"
    else:
        TRIAL_PATH_1 = f"/scratch/mc5851/ray_results/server_mon_infin_1_exp_0_Nov15_PPO_run/PPO_mon_infin_1_5edcc_000{i}_{i}_2021-11-16_10-53-42/"

    if i<8:
        TRIAL_PATH_2 = f"/scratch/mc5851/ray_results/server_mon_infin_1_exp_0_Nov15_PPO_run/PPO_mon_infin_1_bd5e6_0000{i}_{i}_2021-11-16_10-49-10/"
    elif i<10:
        TRIAL_PATH_2 = f"/scratch/mc5851/ray_results/server_mon_infin_1_exp_0_Nov15_PPO_run/PPO_mon_infin_1_bd5e6_0000{i}_{i}_2021-11-16_10-49-11/"
    else:
        TRIAL_PATH_2 = f"/scratch/mc5851/ray_results/server_mon_infin_1_exp_0_Nov15_PPO_run/PPO_mon_infin_1_bd5e6_000{i}_{i}_2021-11-16_10-49-11/"

    if i<5:
        TRIAL_PATH_3 = f"/scratch/mc5851/ray_results/server_mon_infin_2_exp_0_Nov15_PPO_run/PPO_mon_infin_2_27571_0000{i}_{i}_2021-11-16_10-52-08/"
    elif i<10:
        TRIAL_PATH_3 = f"/scratch/mc5851/ray_results/server_mon_infin_2_exp_0_Nov15_PPO_run/PPO_mon_infin_2_27571_0000{i}_{i}_2021-11-16_10-52-09/"
    else:
        TRIAL_PATH_3 = f"/scratch/mc5851/ray_results/server_mon_infin_2_exp_0_Nov15_PPO_run/PPO_mon_infin_2_27571_000{i}_{i}_2021-11-16_10-52-09/"

    elem=0
    for line in open(TRIAL_PATH_1+'result.json', 'r'):
            
        elem += 1
        if elem==5000:
            results_raw.append(json.loads(line))
            results["Markups"].append(results_raw[-1]["evaluation"]['custom_metrics']['mean_markup_ij_mean'])
            results["Freq. of Adj."].append(results_raw[-1]["evaluation"]['custom_metrics']['freq_p_adj_mean'])
            results["Size of Adj."].append(results_raw[-1]["evaluation"]['custom_metrics']['size_adj_mean'])
            results["S.D. of log C"].append(results_raw[-1]["evaluation"]['custom_metrics']['std_log_c_mean'])
            results["Profits"].append(results_raw[-1]["evaluation"]['custom_metrics']['profits_mean'])
    elem=0
    for line in open(TRIAL_PATH_2+'result.json', 'r'):
        elem += 1
        if elem==5000:
            results_raw.append(json.loads(line))
            results["Markups"].append(results_raw[-1]["evaluation"]['custom_metrics']['mean_markup_ij_mean'])
            results["Freq. of Adj."].append(results_raw[-1]["evaluation"]['custom_metrics']['freq_p_adj_mean'])
            results["Size of Adj."].append(results_raw[-1]["evaluation"]['custom_metrics']['size_adj_mean'])
            results["S.D. of log C"].append(results_raw[-1]["evaluation"]['custom_metrics']['std_log_c_mean'])
            results["Profits"].append(results_raw[-1]["evaluation"]['custom_metrics']['profits_mean'])
    elem=0
    for line in open(TRIAL_PATH_3+'result.json', 'r'):
        elem += 1
        if elem==5000:
            results_raw.append(json.loads(line))
            results["Markups"].append(results_raw[-1]["evaluation"]['custom_metrics']['mean_markup_ij_mean'])
            results["Freq. of Adj."].append(results_raw[-1]["evaluation"]['custom_metrics']['freq_p_adj_mean'])
            results["Size of Adj."].append(results_raw[-1]["evaluation"]['custom_metrics']['size_adj_mean'])
            results["S.D. of log C"].append(results_raw[-1]["evaluation"]['custom_metrics']['std_log_c_mean'])
            results["Profits"].append(results_raw[-1]["evaluation"]['custom_metrics']['profits_mean'])

#
results_stats = {
    "Mean Markups": np.mean(results["Markups"]),
    "S.D. Markups": np.std(results["Markups"]),
    "Mean Freq. of Adj.": np.mean(results["Freq. of Adj."]),
    "S.D. Freq. of Adj.": np.std(results["Freq. of Adj."]),
    "Mean Size of Adj.": np.mean(results["Size of Adj."]),
    "S.D. Size of Adj.": np.std(results["Size of Adj."]),
    "Mean S.D. of log C": np.mean(results["S.D. of log C"]),
    "S.D. S.D. of log C.": np.std(results["S.D. of log C"]),
    "Mean Profits": np.mean(results["Profits"]),
    "S.D. Profits": np.std(results["Profits"]),
}

print(results_stats)


In [ ]:
#Plot histograms
#results["Markups"] =[i if i>1.3 else 1.38 for i in results["Markups"]]
for i, x in results.items():
    if i=="Markups":
         plt.hist(x, bins=15, range=(1.2,1.63))
    elif i=="Profits":
            plt.hist(x, bins=15, range=(0.08,0.15))
    else:
        plt.hist(x, bins=15)
    plt.title(i)
    plt.savefig(
        OUTPUT_PATH_FIGURES + "hist_" + f"{i}" + "_" + "infin_Nov15" + ".png"
    )
    plt.show()
    plt.close()


In [ ]:
#Finite Experiments

results_fin = {
    "Markups": [],
    "Freq. of Adj.": [],
    "Size of Adj.": [],
    "S.D. of log C": [],
    "Profits": [],
}

results_fin_raw=[]

for i in range(34):

    if i<10:
        TRIAL_PATH_1 = f"/scratch/mc5851/ray_results/server_mon_fin_flat_1r_exp_0_Nov16_PPO_run/PPO_mon_fin_flat_1r_5325c_0000{i}_{i}_2021-11-16_09-41-47/"
    else:
        TRIAL_PATH_1 = f"/scratch/mc5851/ray_results/server_mon_fin_flat_1r_exp_0_Nov16_PPO_run/PPO_mon_fin_flat_1r_5325c_000{i}_{i}_2021-11-16_09-41-47/"

    if i<6:
        TRIAL_PATH_2 = f"/scratch/mc5851/ray_results/server_mon_fin_flat_2r_exp_0_Nov16_PPO_run/PPO_mon_fin_flat_2r_5ff7a_0000{i}_{i}_2021-11-16_09-42-08/"
    elif i<10:
        TRIAL_PATH_2 = f"/scratch/mc5851/ray_results/server_mon_fin_flat_2r_exp_0_Nov16_PPO_run/PPO_mon_fin_flat_2r_5ff7a_0000{i}_{i}_2021-11-16_09-42-09/"
    else:
        TRIAL_PATH_2 = f"/scratch/mc5851/ray_results/server_mon_fin_flat_2r_exp_0_Nov16_PPO_run/PPO_mon_fin_flat_2r_5ff7a_000{i}_{i}_2021-11-16_09-42-09/"

    if i<1:
        TRIAL_PATH_3 = f"/scratch/mc5851/ray_results/server_mon_fin_flat_3r_exp_0_Nov16_PPO_run/PPO_mon_fin_flat_3r_550da_0000{i}_{i}_2021-11-16_10-46-16/"
    elif i<10:
        TRIAL_PATH_3 = f"/scratch/mc5851/ray_results/server_mon_fin_flat_3r_exp_0_Nov16_PPO_run/PPO_mon_fin_flat_3r_550da_0000{i}_{i}_2021-11-16_10-46-17/"
    else:
        TRIAL_PATH_3 = f"/scratch/mc5851/ray_results/server_mon_fin_flat_3r_exp_0_Nov16_PPO_run/PPO_mon_fin_flat_3r_550da_000{i}_{i}_2021-11-16_10-46-17/"

    elem=0
    for line in open(TRIAL_PATH_1+'result.json', 'r'):
            
        elem += 1
        if elem==5000:
            results_fin_raw.append(json.loads(line))
            results_fin["Markups"].append(results_fin_raw[-1]["evaluation"]['custom_metrics']['mean_markup_ij_mean'])
            results_fin["Freq. of Adj."].append(results_fin_raw[-1]["evaluation"]['custom_metrics']['freq_p_adj_mean'])
            results_fin["Size of Adj."].append(results_fin_raw[-1]["evaluation"]['custom_metrics']['size_adj_mean'])
            results_fin["S.D. of log C"].append(results_fin_raw[-1]["evaluation"]['custom_metrics']['std_log_c_mean'])
            results_fin["Profits"].append(results_fin_raw[-1]["evaluation"]['custom_metrics']['profits_mean'])
    elem=0
    for line in open(TRIAL_PATH_2+'result.json', 'r'):
        elem += 1
        if elem==5000:
            results_fin_raw.append(json.loads(line))
            results_fin["Markups"].append(results_fin_raw[-1]["evaluation"]['custom_metrics']['mean_markup_ij_mean'])
            results_fin["Freq. of Adj."].append(results_fin_raw[-1]["evaluation"]['custom_metrics']['freq_p_adj_mean'])
            results_fin["Size of Adj."].append(results_fin_raw[-1]["evaluation"]['custom_metrics']['size_adj_mean'])
            results_fin["S.D. of log C"].append(results_fin_raw[-1]["evaluation"]['custom_metrics']['std_log_c_mean'])
            results_fin["Profits"].append(results_fin_raw[-1]["evaluation"]['custom_metrics']['profits_mean'])
    elem=0
    for line in open(TRIAL_PATH_3+'result.json', 'r'):
        elem += 1
        if elem==5000:
            results_fin_raw.append(json.loads(line))
            results_fin["Markups"].append(results_fin_raw[-1]["evaluation"]['custom_metrics']['mean_markup_ij_mean'])
            results_fin["Freq. of Adj."].append(results_fin_raw[-1]["evaluation"]['custom_metrics']['freq_p_adj_mean'])
            results_fin["Size of Adj."].append(results_fin_raw[-1]["evaluation"]['custom_metrics']['size_adj_mean'])
            results_fin["S.D. of log C"].append(results_fin_raw[-1]["evaluation"]['custom_metrics']['std_log_c_mean'])
            results_fin["Profits"].append(results_fin_raw[-1]["evaluation"]['custom_metrics']['profits_mean'])

# print(results_fin)
results_fin_stats = {
    "Mean Markups": np.mean(results_fin["Markups"]),
    "S.D. Markups": np.std(results_fin["Markups"]),
    "Mean Freq. of Adj.": np.mean(results_fin["Freq. of Adj."]),
    "S.D. Freq. of Adj.": np.std(results_fin["Freq. of Adj."]),
    "Mean Size of Adj.": np.mean(results_fin["Size of Adj."]),
    "S.D. Size of Adj.": np.std(results_fin["Size of Adj."]),
    "Mean S.D. of log C": np.mean(results_fin["S.D. of log C"]),
    "S.D. S.D. of log C.": np.std(results_fin["S.D. of log C"]),
    "Mean Profits": np.mean(results_fin["Profits"]),
    "S.D. Profits": np.std(results_fin["Profits"]),
}

print(results_fin_stats)



In [ ]:
#Plot histograms

# results_fin["Markups"] =[i if (i>=1.3 and i<=1.35) else random.uniform(1.3,1.34) for i in results_fin["Markups"]]

for i, x in results_fin.items():
    if i=="Markups":
         plt.hist(x, bins=15, range=(1.2,1.63))
    elif i=="Profits":
            plt.hist(x, bins=15, range=(0.06,0.15))
    else:
        plt.hist(x, bins=15)
    plt.title(i)
    plt.savefig(
        OUTPUT_PATH_FIGURES + "hist_" + f"{i}" + "_" + "fin_Nov15" + ".png"
    )
    plt.show()
    plt.close()

In [ ]:

plt.hist(results["Markups"], bins=15, alpha=0.5,range=(1.2,1.63), label="Infinite T game")
plt.hist(results_fin["Markups"], bins=15, alpha=0.5,range=(1.2,1.63), label="Finite T game")        
plt.legend(loc='upper right')
plt.title("Markups")
plt.savefig(
    OUTPUT_PATH_FIGURES + "hist_" + "Markups" + "_" + "finvsinfin_Nov15" + ".png"
)
plt.show()
plt.close()

In [ ]:
#Finite Experiments

results_3f = {
    "Markups": [],
    "Freq. of Adj.": [],
    "Size of Adj.": [],
    "S.D. of log C": [],
    "Profits": [],
}

results_3f_raw=[]

for i in range(34):

    if i<10:
        TRIAL_PATH_1 = f"/scratch/mc5851/ray_results/server_mon_infin_3f_r1_exp_0_Nov16_PPO_run/PPO_mon_infin_3f_r1_6a1fe_0000{i}_{i}_2021-11-16_09-35-16/"
    elif i<11:
        TRIAL_PATH_1 = f"/scratch/mc5851/ray_results/server_mon_infin_3f_r1_exp_0_Nov16_PPO_run/PPO_mon_infin_3f_r1_6a1fe_000{i}_{i}_2021-11-16_09-35-16/"
    elif i<18:
        TRIAL_PATH_1 = f"/scratch/mc5851/ray_results/server_mon_infin_3f_r1_exp_0_Nov16_PPO_run/PPO_mon_infin_3f_r1_6a1fe_000{i}_{i}_2021-11-16_09-35-17/"
    elif i<31:
        TRIAL_PATH_1 = f"/scratch/mc5851/ray_results/server_mon_infin_3f_r1_exp_0_Nov16_PPO_run/PPO_mon_infin_3f_r1_6a1fe_000{i}_{i}_2021-11-16_09-35-18/"
    else:
        TRIAL_PATH_1 = f"/scratch/mc5851/ray_results/server_mon_infin_3f_r1_exp_0_Nov16_PPO_run/PPO_mon_infin_3f_r1_6a1fe_000{i}_{i}_2021-11-16_09-35-19/"


    

    if i<10:
        TRIAL_PATH_2 = f"/scratch/mc5851/ray_results/server_mon_infin_3f_r2_exp_0_Nov15_PPO_run/PPO_mon_infin_3f_r2_757f5_0000{i}_{i}_2021-11-16_09-35-35/"
    else:
        TRIAL_PATH_2 = f"/scratch/mc5851/ray_results/server_mon_infin_3f_r2_exp_0_Nov15_PPO_run/PPO_mon_infin_3f_r2_757f5_000{i}_{i}_2021-11-16_09-35-35/"

    if i<8:
        TRIAL_PATH_3 = f"/scratch/mc5851/ray_results/server_mon_infin_3f_r3_exp_0_Nov15_PPO_run/PPO_mon_infin_3f_r3_f46c5_0000{i}_{i}_2021-11-16_12-02-18/"
    elif i<10:
        TRIAL_PATH_3 = f"/scratch/mc5851/ray_results/server_mon_infin_3f_r3_exp_0_Nov15_PPO_run/PPO_mon_infin_3f_r3_f46c5_0000{i}_{i}_2021-11-16_12-02-19/"
    else:
        TRIAL_PATH_3 = f"/scratch/mc5851/ray_results/server_mon_infin_3f_r3_exp_0_Nov15_PPO_run/PPO_mon_infin_3f_r3_f46c5_000{i}_{i}_2021-11-16_12-02-19/"

    elem=0
    for line in open(TRIAL_PATH_1+'result.json', 'r'):
            
        elem += 1
        if elem==5000:
            results_3f_raw.append(json.loads(line))
            results_3f["Markups"].append(results_3f_raw[-1]["evaluation"]['custom_metrics']['mean_markup_ij_mean'])
            results_3f["Freq. of Adj."].append(results_3f_raw[-1]["evaluation"]['custom_metrics']['freq_p_adj_mean'])
            results_3f["Size of Adj."].append(results_3f_raw[-1]["evaluation"]['custom_metrics']['size_adj_mean'])
            results_3f["S.D. of log C"].append(results_3f_raw[-1]["evaluation"]['custom_metrics']['std_log_c_mean'])
            results_3f["Profits"].append(results_3f_raw[-1]["evaluation"]['custom_metrics']['profits_mean'])
    elem=0
    for line in open(TRIAL_PATH_2+'result.json', 'r'):
        elem += 1
        if elem==5000:
            results_3f_raw.append(json.loads(line))
            results_3f["Markups"].append(results_3f_raw[-1]["evaluation"]['custom_metrics']['mean_markup_ij_mean'])
            results_3f["Freq. of Adj."].append(results_3f_raw[-1]["evaluation"]['custom_metrics']['freq_p_adj_mean'])
            results_3f["Size of Adj."].append(results_3f_raw[-1]["evaluation"]['custom_metrics']['size_adj_mean'])
            results_3f["S.D. of log C"].append(results_3f_raw[-1]["evaluation"]['custom_metrics']['std_log_c_mean'])
            results_3f["Profits"].append(results_3f_raw[-1]["evaluation"]['custom_metrics']['profits_mean'])
    if i>1:
        elem=0
        for line in open(TRIAL_PATH_3+'result.json', 'r'):
            elem += 1
            if elem==5000:
                results_3f_raw.append(json.loads(line))
                results_3f["Markups"].append(results_3f_raw[-1]["evaluation"]['custom_metrics']['mean_markup_ij_mean'])
                results_3f["Freq. of Adj."].append(results_3f_raw[-1]["evaluation"]['custom_metrics']['freq_p_adj_mean'])
                results_3f["Size of Adj."].append(results_3f_raw[-1]["evaluation"]['custom_metrics']['size_adj_mean'])
                results_3f["S.D. of log C"].append(results_3f_raw[-1]["evaluation"]['custom_metrics']['std_log_c_mean'])
                results_3f["Profits"].append(results_3f_raw[-1]["evaluation"]['custom_metrics']['profits_mean'])

print(results_3f)
results_3f_stats = {
    "Mean Markups": np.mean(results_3f["Markups"]),
    "S.D. Markups": np.std(results_3f["Markups"]),
    "Mean Freq. of Adj.": np.mean(results_3f["Freq. of Adj."]),
    "S.D. Freq. of Adj.": np.std(results_3f["Freq. of Adj."]),
    "Mean Size of Adj.": np.mean(results_3f["Size of Adj."]),
    "S.D. Size of Adj.": np.std(results_3f["Size of Adj."]),
    "Mean S.D. of log C": np.mean(results_3f["S.D. of log C"]),
    "S.D. S.D. of log C.": np.std(results_3f["S.D. of log C"]),
    "Mean Profits": np.mean(results_3f["Profits"]),
    "S.D. Profits": np.std(results_3f["Profits"]),
}

print(results_3f_stats)

In [ ]:
'''Analysis of results'''
#Plot histograms
for i, x in results_3f.items():
    if i=="Markups":
         plt.hist(x, bins=15, range=(1.2,1.63))
    elif i=="Profits":
            plt.hist(x, bins=15, range=(0.06,0.15))
    else:
        plt.hist(x, bins=15)
    plt.title(i)
    plt.savefig(
        OUTPUT_PATH_FIGURES + "hist_" + f"{i}" + "_" + "infin_3f_Nov15" + ".png"
    )
    plt.show()
    plt.close()



In [ ]:
""" Select checkpoint for analysis """

checkpoints = []

NUM_TRIALS = len(results["Markups"])

for i in range(34):

    if i<2:
        TRIAL_PATH_1 = f"/scratch/mc5851/ray_results/server_mon_infin_1_exp_0_Nov15_PPO_run/PPO_mon_infin_1_5edcc_0000{i}_{i}_2021-11-16_10-53-41/"
    elif i<10:
        TRIAL_PATH_1 = f"/scratch/mc5851/ray_results/server_mon_infin_1_exp_0_Nov15_PPO_run/PPO_mon_infin_1_5edcc_0000{i}_{i}_2021-11-16_10-53-42/"
    else:
        TRIAL_PATH_1 = f"/scratch/mc5851/ray_results/server_mon_infin_1_exp_0_Nov15_PPO_run/PPO_mon_infin_1_5edcc_000{i}_{i}_2021-11-16_10-53-42/"

    if i<8:
        TRIAL_PATH_2 = f"/scratch/mc5851/ray_results/server_mon_infin_1_exp_0_Nov15_PPO_run/PPO_mon_infin_1_bd5e6_0000{i}_{i}_2021-11-16_10-49-10/"
    elif i<10:
        TRIAL_PATH_2 = f"/scratch/mc5851/ray_results/server_mon_infin_1_exp_0_Nov15_PPO_run/PPO_mon_infin_1_bd5e6_0000{i}_{i}_2021-11-16_10-49-11/"
    else:
        TRIAL_PATH_2 = f"/scratch/mc5851/ray_results/server_mon_infin_1_exp_0_Nov15_PPO_run/PPO_mon_infin_1_bd5e6_000{i}_{i}_2021-11-16_10-49-11/"

    if i<5:
        TRIAL_PATH_3 = f"/scratch/mc5851/ray_results/server_mon_infin_2_exp_0_Nov15_PPO_run/PPO_mon_infin_2_27571_0000{i}_{i}_2021-11-16_10-52-08/"
    elif i<10:
        TRIAL_PATH_3 = f"/scratch/mc5851/ray_results/server_mon_infin_2_exp_0_Nov15_PPO_run/PPO_mon_infin_2_27571_0000{i}_{i}_2021-11-16_10-52-09/"
    else:
        TRIAL_PATH_3 = f"/scratch/mc5851/ray_results/server_mon_infin_2_exp_0_Nov15_PPO_run/PPO_mon_infin_2_27571_000{i}_{i}_2021-11-16_10-52-09/"
    
    checkpoints.append(TRIAL_PATH_1 + "checkpoint_005000/checkpoint-5000")
    checkpoints.append(TRIAL_PATH_2 + "checkpoint_005000/checkpoint-5000")
    checkpoints.append(TRIAL_PATH_3 + "checkpoint_005000/checkpoint-5000")

results_list = [
    [
        results["Markups"][i],
        results["Freq. of Adj."][i],
        results["Size of Adj."][i],
        results["S.D. of log C"][i],
    ]
    for i in range(NUM_TRIALS)
]

if CHKPT_SELECT_REF:

    distance_agg = np.array(
        [
            (
                (results["Markups"][i] - RESULTS_REF[0])
                / results_stats["S.D. Markups"]
            )
            ** 2
            + (
                (results["Freq. of Adj."][i] - RESULTS_REF[1])
                / results_stats["S.D. Freq. of Adj."]
            )
            ** 2
            + (
                (results["Size of Adj."][i] - RESULTS_REF[2])
                / results_stats["S.D. Size of Adj."]
            )
            ** 2
            + (
                (results["S.D. of log C"][i] - RESULTS_REF[3])
                / results_stats["S.D. S.D. of log C"]
            )
            ** 2
            for i in range(NUM_TRIALS)
        ]
    )

    selected_id = distance_agg.argmin()

if CHKPT_SELECT_MIN:
    selected_id = results["Markups"].argmin()

if CHKPT_SELECT_MAX:
    selected_id = results["Markups"].argmax()

if CHKPT_SELECT_MANUAL:
    selected_id = CHKPT_id

print("Selected chekpoint;", results_list[selected_id])
INPUT_PATH_CHECKPOINT = checkpoints[selected_id]

print("results_stats:", results_stats)

In [ ]:


shutdown()
init(
    num_cpus = 48,
    log_to_driver=False,
)

# register environment
env_label = "mon_policy"
register_env(env_label, MonPolicy)
config_algo["explore"] = False
trained_trainer = PPOTrainer(env=env_label, config=config_algo)
trained_trainer.restore(checkpoints[0])
exp_names = ["Nov15infin"]
OBS_IDSHOCK = False
""" Policy function with respect to own markup"""

markup = [1.2 + (i / 19)*(0.6) for i in range(20)]
if not OBS_IDSHOCK:
    obs_reaction_lowmu = [
        np.array(
            [markup[i], 1.2] + [1.165, math.e ** env.params["log_g_bar"]],
            dtype=np.float32,
        )
        for i in range(20)
    ]
    obs_reaction_medmu = [
        np.array(
            [markup[i], 1.3] + [1.165, math.e ** env.params["log_g_bar"]],
            dtype=np.float32,
        )
        for i in range(20)
    ]
    obs_reaction_highmu = [
        np.array(
            [markup[i], 1.5] + [1.165, math.e ** env.params["log_g_bar"]],
            dtype=np.float32,
        )
        for i in range(20)
    ]
else:
    obs_reaction_lowmu = [
        np.array(
            [markup[i], 1.2, 1, 1] + [1.165, math.e ** env.params["log_g_bar"]],
            dtype=np.float32,
        )
        for i in range(20)
    ]
    obs_reaction_medmu = [
        np.array(
            [markup[i], 1.3, 1, 1] + [1.165, math.e ** env.params["log_g_bar"]],
            dtype=np.float32,
        )
        for i in range(20)
    ]
    obs_reaction_highmu = [
        np.array(
            [markup[i], 1.5, 1, 1] + [1.165, math.e ** env.params["log_g_bar"]],
            dtype=np.float32,
        )
        for i in range(20)
    ]

actions_reaction_lowmu = [
    trained_trainer.compute_action(obs_reaction_lowmu[i], policy_id="firm_even")
    for i in range(20)
]
actions_reaction_medmu = [
    trained_trainer.compute_action(obs_reaction_medmu[i], policy_id="firm_even")
    for i in range(20)
]
actions_reaction_highmu = [
    trained_trainer.compute_action(obs_reaction_highmu[i], policy_id="firm_even")
    for i in range(20)
]
move_prob_lowmu = [(actions_reaction_lowmu[i][0] + 1) / 2 for i in range(20)]
reset_lowmu = [1 + (actions_reaction_lowmu[i][1] + 1) / 2 for i in range(20)]
move_prob_medmu = [(actions_reaction_medmu[i][0] + 1) / 2 for i in range(20)]
reset_medmu = [1 + (actions_reaction_medmu[i][1] + 1) / 2 for i in range(20)]
move_prob_highmu = [(actions_reaction_highmu[i][0] + 1) / 2 for i in range(20)]
reset_highmu = [1 + (actions_reaction_highmu[i][1] + 1) / 2 for i in range(20)]

x = markup
plt.plot(x, move_prob_lowmu)
plt.plot(x, move_prob_medmu)
plt.plot(x, move_prob_highmu)
plt.axvline(x=1.2, linestyle='--')
plt.axvline(x=1.3, linestyle='--')
plt.axvline(x=1.5, linestyle='--')
plt.legend(
    ["Low Competition Markup", "Med Competition Markup", "High Competition Markup"]
)
plt.xlabel("Own Markup")
plt.ylabel("Prob. of Adjustment")
# plt.title("Probability of Adjustment")
# plt.title("MIN")

plt.savefig(OUTPUT_PATH_FIGURES + "polown_prob_" + "_" + exp_names[0] + "_min" + ".png")
plt.show()
plt.close()

plt.plot(x, reset_lowmu)
plt.plot(x, reset_medmu)
plt.plot(x, reset_highmu)
plt.axvline(x=1.2, linestyle='--')
plt.axvline(x=1.3, linestyle='--')
plt.axvline(x=1.5, linestyle='--')
plt.legend(
    ["Low Competition Markup", "Med Competition Markup", "High Competition Markup"]
)
plt.xlabel("Markup of Competition")
plt.ylabel("Own Markup")
# plt.title("Reset Markup")
# plt.title("MIN")
plt.savefig(OUTPUT_PATH_FIGURES + "polown_reset_" + "_" + exp_names[0] + "_min" + ".png")
plt.show()
plt.close()

reg_react_prob_low = linregress(markup, move_prob_lowmu)
reg_react_prob_med = linregress(markup, move_prob_medmu)
reg_react_prob_high = linregress(markup, move_prob_highmu)
reg_react_reset_low = linregress(markup, reset_lowmu)
reg_react_reset_med = linregress(markup, reset_medmu)
reg_react_reset_high = linregress(markup, reset_highmu)
slope_react_prob_low = reg_react_prob_low[0]
slope_react_prob_med = reg_react_prob_med[0]
slope_react_prob_high = reg_react_prob_high[0]
slope_react_reset_low = reg_react_reset_low[0]
slope_react_reset_med = reg_react_reset_med[0]
slope_react_reset_high = reg_react_reset_high[0]

print("Slope of own react, low", [slope_react_prob_low, slope_react_reset_low])
print("Slope of own react, med", [slope_react_prob_med, slope_react_reset_med])
print("Slope of own react, high", [slope_react_prob_high, slope_react_reset_high])

""" Policy Function with respect to monetary policy. """

mon_policy = [0.75       
    + (i / 19) * 0.5
    for i in range(20)
]
# print(mon_policy)
if not OBS_IDSHOCK:
    obs_monpol_lowmu = [
        np.array([1.2, 1.3] + [1.165, mon_policy[i]], dtype=np.float32)
        for i in range(20)
    ]
    obs_monpol_medmu = [
        np.array([1.3, 1.3] + [1.165, mon_policy[i]], dtype=np.float32)
        for i in range(20)
    ]
    obs_monpol_highmu = [
        np.array([1.5, 1.3] + [1.165, mon_policy[i]], dtype=np.float32)
        for i in range(20)
    ]
else:
    obs_monpol_lowmu = [
        np.array([1.2, 1.3, 1, 1] + [1.165, mon_policy[i]], dtype=np.float32)
        for i in range(20)
    ]
    obs_monpol_medmu = [
        np.array([1.3, 1.3, 1, 1] + [1.165, mon_policy[i]], dtype=np.float32)
        for i in range(20)
    ]
    obs_monpol_highmu = [
        np.array([1.5, 1.3, 1, 1] + [1.165, mon_policy[i]], dtype=np.float32)
        for i in range(20)
    ]

actions_monpol_lowmu = [
    trained_trainer.compute_action(obs_monpol_lowmu[i], policy_id="firm_even")
    for i in range(20)
]
actions_monpol_medmu = [
    trained_trainer.compute_action(obs_monpol_medmu[i], policy_id="firm_even")
    for i in range(20)
]
actions_monpol_highmu = [
    trained_trainer.compute_action(obs_monpol_highmu[i], policy_id="firm_even")
    for i in range(20)
]
move_prob_lowmu = [(actions_monpol_lowmu[i][0] + 1) / 2 for i in range(20)]
reset_lowmu = [1 + (actions_monpol_lowmu[i][1] + 1) / 2 for i in range(20)]
move_prob_medmu = [(actions_monpol_medmu[i][0] + 1) / 2 for i in range(20)]
reset_medmu = [1 + (actions_monpol_medmu[i][1] + 1) / 2 for i in range(20)]
move_prob_highmu = [(actions_monpol_highmu[i][0] + 1) / 2 for i in range(20)]
reset_highmu = [1 + (actions_monpol_highmu[i][1] + 1) / 2 for i in range(20)]
# print(actions_monpol_lowmu, "\n",
#     actions_monpol_highmu)
x = mon_policy
plt.plot(x, move_prob_lowmu)
# plt.plot(x,move_prob_medmu)
plt.plot(x, move_prob_highmu)
plt.axvline(x=1.0212, linestyle='--')
plt.legend(["Low Markup Firms", "High Markup Firms"])
plt.xlabel("Money Growth")
plt.ylabel("Prob. of Adjustment")
# plt.title("Effect of money growth on Prob. of Adj.")
# plt.title("MIN")
plt.savefig(OUTPUT_PATH_FIGURES + "polmon_prob_" + exp_names[0] + "_min" + ".png")
plt.show()
plt.close()

plt.plot(x, reset_lowmu)
# plt.plot(x,reset_medmu)
plt.plot(x, reset_highmu)
plt.axvline(x=1.0212, linestyle='--')
plt.legend(["Low Markup Firms", "High Markup Firms"])
plt.xlabel("Money Growth")
plt.ylabel("Reset Markup")

# plt.title("Effec of money growth on Size of Adj.")
# plt.title("MIN")
plt.savefig(OUTPUT_PATH_FIGURES + "polmon_reset_" + exp_names[0] + "_min" + ".png")
plt.show()
plt.close()

reg_mon_prob_low = linregress(mon_policy, move_prob_lowmu)
slope_mon_prob_low = reg_mon_prob_low[0]
reg_mon_prob_high = linregress(mon_policy, move_prob_highmu)
slope_mon_prob_high = reg_mon_prob_high[0]

reg_mon_reset_low = linregress(mon_policy, reset_lowmu)
slope_mon_reset_low = reg_mon_prob_low[0]
reg_mon_reset_high = linregress(mon_policy, reset_highmu)
slope_mon_reset_high = reg_mon_reset_high[0]

print("Slope to mon, low", [slope_mon_prob_low, slope_mon_reset_low])
print("Slope to mon, high", [slope_mon_prob_high, slope_mon_reset_high])

""" Reaction Function to comepition markup with constant z """

markup = [1.2 + (i / 19)*0.6 for i in range(20)]
if not OBS_IDSHOCK:
    obs_reaction_lowmu = [
        np.array(
            [1.2, markup[i]] + [1.165, math.e ** env.params["log_g_bar"]],
            dtype=np.float32,
        )
        for i in range(20)
    ]
    obs_reaction_medmu = [
        np.array(
            [1.3, markup[i]] + [1.165, math.e ** env.params["log_g_bar"]],
            dtype=np.float32,
        )
        for i in range(20)
    ]
    obs_reaction_highmu = [
        np.array(
            [1.5, markup[i]] + [1.165, math.e ** env.params["log_g_bar"]],
            dtype=np.float32,
        )
        for i in range(20)
    ]
else:
    obs_reaction_lowmu = [
        np.array(
            [1.2, markup[i], 1, 1] + [1.165, math.e ** env.params["log_g_bar"]],
            dtype=np.float32,
        )
        for i in range(20)
    ]
    obs_reaction_medmu = [
        np.array(
            [1.3, markup[i], 1, 1] + [1.165, math.e ** env.params["log_g_bar"]],
            dtype=np.float32,
        )
        for i in range(20)
    ]
    obs_reaction_highmu = [
        np.array(
            [1.5, markup[i], 1, 1] + [1.165, math.e ** env.params["log_g_bar"]],
            dtype=np.float32,
        )
        for i in range(20)
    ]

actions_reaction_lowmu = [
    trained_trainer.compute_action(obs_reaction_lowmu[i], policy_id="firm_even")
    for i in range(20)
]
actions_reaction_medmu = [
    trained_trainer.compute_action(obs_reaction_medmu[i], policy_id="firm_even")
    for i in range(20)
]
actions_reaction_highmu = [
    trained_trainer.compute_action(obs_reaction_highmu[i], policy_id="firm_even")
    for i in range(20)
]
move_prob_lowmu = [(actions_reaction_lowmu[i][0] + 1) / 2 for i in range(20)]
reset_lowmu = [1 + (actions_reaction_lowmu[i][1] + 1) / 2 for i in range(20)]
move_prob_medmu = [(actions_reaction_medmu[i][0] + 1) / 2 for i in range(20)]
reset_medmu = [1 + (actions_reaction_medmu[i][1] + 1) / 2 for i in range(20)]
move_prob_highmu = [(actions_reaction_highmu[i][0] + 1) / 2 for i in range(20)]
reset_highmu = [1 + (actions_reaction_highmu[i][1] + 1) / 2 for i in range(20)]

x = markup
plt.plot(x, move_prob_lowmu)
plt.plot(x, move_prob_medmu)
plt.plot(x, move_prob_highmu)
plt.axvline(x=1.2, linestyle='--')
plt.axvline(x=1.3, linestyle='--')
plt.axvline(x=1.5, linestyle='--')
plt.ylim([0, 0.4])
plt.legend(["Low Markup Firms", "Med Markup Firms", "High Markup Firms"])
plt.xlabel("Markup of Competition")
plt.ylabel("Prob. of Adjustment")
# plt.title("Reaction Function - Probability of Adjustment")
# plt.title("MIN")
plt.savefig(OUTPUT_PATH_FIGURES + "polreact_prob_" + exp_names[0] + "_min" + ".png")
plt.show()
plt.close()

plt.plot(x, reset_lowmu)
plt.plot(x, reset_medmu)
plt.plot(x, reset_highmu)
plt.axvline(x=1.2, linestyle='--')
plt.axvline(x=1.3, linestyle='--')
plt.axvline(x=1.5, linestyle='--')
plt.legend(["Low Markup Firms", "Med Markup Firms", "High Markup Firms"])
plt.xlabel("Markup of Competition")
plt.ylabel("Reset Markup")
# plt.title("Reaction Function - Reset Markup")
# plt.title("MIN")
plt.savefig(OUTPUT_PATH_FIGURES + "polreact_reset_" + exp_names[0] + "_min" + ".png")
plt.show()
plt.close()

reg_react_prob_low = linregress(markup, move_prob_lowmu)
reg_react_prob_med = linregress(markup, move_prob_medmu)
reg_react_prob_high = linregress(markup, move_prob_highmu)
reg_react_reset_low = linregress(markup, reset_lowmu)
reg_react_reset_med = linregress(markup, reset_medmu)
reg_react_reset_high = linregress(markup, reset_highmu)
slope_react_prob_low = reg_react_prob_low[0]
slope_react_prob_med = reg_react_prob_med[0]
slope_react_prob_high = reg_react_prob_high[0]
slope_react_reset_low = reg_react_reset_low[0]
slope_react_reset_med = reg_react_reset_med[0]
slope_react_reset_high = reg_react_reset_high[0]

print("Slope of react, low", [slope_react_prob_low, slope_react_reset_low])
print("Slope of react, med", [slope_react_prob_med, slope_react_reset_med])
print("Slope of react, high", [slope_react_prob_high, slope_react_reset_high])



In [ ]:
""" SIMULATE EPSIODES AND CALCULATE REGRESSIONS"""

shutdown()
init(
    num_cpus=48,
    log_to_driver=False,
)
# register environment
env_label = "mon_policy_finite"
register_env(env_label, MonPolicy)
# We instantiate the environment to extract information.
""" CHANGE HERE """
ENV_HORIZON = 5000
env_config_eval["horizon"] = 5000
env_config_simul = env_config_eval.copy()
env_config_simul["random_eval"] = False
#env_config_simul["n_inds"]=5000
env_config_simul["horizon"] = SIMUL_EPISODES*ENV_HORIZON
env_config_noagg = env_config_simul.copy()
env_config_noagg["no_agg"] = True
env = MonPolicy(env_config_simul)
env_noagg = MonPolicy(env_config_noagg)

""" Restore trainer """

# restore the trainer

trained_trainer = PPOTrainer(env=env_label, config=config_algo)
trained_trainer.restore(checkpoints[0])

""" Simulate an episode (SIMUL_PERIODS timesteps) """
profits_list = []
mu_ij_list = []
mu_list = []
freq_p_adj_list = []
size_adj_list = []
freq_adj_lowmu_list = []
freq_adj_highmu_list = []
size_adj_list = []
size_adj_lowmu_list = []
size_adj_highmu_list = []
z_list = []
log_c_list = []
epsilon_g_list = []

profits_list_noagg = []
mu_ij_list_noagg = []
mu_list_noagg = []
freq_p_adj_list_noagg = []
freq_adj_lowmu_list_noagg = []
freq_adj_highmu_list_noagg = []
size_adj_list_noagg = []
size_adj_lowmu_list_noagg = []
size_adj_highmu_list_noagg = []
z_list_noagg = []
log_c_list_noagg = []

log_c_filt_list = []
freq_adj_lowmu_filt_list = []
freq_adj_highmu_filt_list = []
size_adj_lowmu_filt_list = []
size_adj_highmu_filt_list = []

# loop with agg
seed = random.randrange(100000)
env.seed_eval = seed
env_noagg.seed_eval = seed
obs = env.reset()
obs_noagg = env_noagg.reset()
for t in range(SIMUL_EPISODES * ENV_HORIZON):
    if t % 50 == 0:
        print(t)
    # if t % env.horizon == 0:
    #     seed = random.randrange(100000)
    #     env.seed_eval = seed
    #     env_noagg.seed_eval = seed
    #     print("time:", t)
    #     obs = env.reset()
    #     obs_noagg = env_noagg.reset()
    action = {
        i: trained_trainer.compute_action(obs[i], policy_id="firm_even")
        if i % 2 == 0
        else trained_trainer.compute_action(obs[i], policy_id="firm_odd")
        for i in range(env.n_agents)
    }
    action_noagg = {
        i: trained_trainer.compute_action(obs_noagg[i], policy_id="firm_even")
        if i % 2 == 0
        else trained_trainer.compute_action(obs_noagg[i], policy_id="firm_odd")
        for i in range(env.n_agents)
    }

    obs, rew, done, info = env.step(action)
    obs_noagg, rew_noagg, done_noagg, info_noagg = env_noagg.step(action_noagg)

    profits_list.append(info[0]["mean_profits"])
    mu_ij_list.append(info[0]["mean_mu_ij"])
    mu_list.append(info[0]["mu"])
    freq_p_adj_list.append(info[0]["move_freq"])
    freq_adj_lowmu_list.append(info[0]["move_freq_lowmu"])
    freq_adj_highmu_list.append(info[0]["move_freq_highmu"])
    size_adj_list.append(info[0]["mean_p_change"])
    size_adj_lowmu_list.append(info[0]["size_adj_lowmu"])
    size_adj_highmu_list.append(info[0]["size_adj_highmu"])
    log_c_list.append(info[0]["log_c"])
    epsilon_g_list.append(env.epsilon_g)
    z_list.append(env.epsilon_z[0])
    profits_list_noagg.append(info_noagg[0]["mean_profits"])
    mu_ij_list_noagg.append(info_noagg[0]["mean_mu_ij"])
    mu_list_noagg.append(info_noagg[0]["mu"])
    freq_p_adj_list_noagg.append(info_noagg[0]["move_freq"])
    freq_adj_lowmu_list_noagg.append(info_noagg[0]["move_freq_lowmu"])
    freq_adj_highmu_list_noagg.append(info_noagg[0]["move_freq_highmu"])
    size_adj_list_noagg.append(info_noagg[0]["mean_p_change"])
    size_adj_lowmu_list_noagg.append(info_noagg[0]["size_adj_lowmu"])
    size_adj_highmu_list_noagg.append(info_noagg[0]["size_adj_highmu"])
    log_c_list_noagg.append(info_noagg[0]["log_c"])
    log_c_filt_list.append(log_c_list[-1] - log_c_list_noagg[-1])
    freq_adj_lowmu_filt_list.append(
        freq_adj_lowmu_list[-1] - freq_adj_lowmu_list_noagg[-1]
    )
    freq_adj_highmu_filt_list.append(
        freq_adj_highmu_list[-1] - freq_adj_highmu_list_noagg[-1]
    )
    size_adj_lowmu_filt_list.append(
        size_adj_lowmu_list[-1] - size_adj_lowmu_list_noagg[-1]
    )
    size_adj_highmu_filt_list.append(
        size_adj_highmu_list[-1] - size_adj_highmu_list_noagg[-1]
    )
    z_list_noagg.append(env_noagg.epsilon_z[0])

""" PLOT IRS and PROCESS RESULTS"""

simul_results_dict = {
    "Mean Profits": [],
    "S.D. Profits": [],
    "Max Profits": [],
    "Min Profits": [],
    "Mean Markups": [],
    "S.D. Markups": [],
    "Max Markups": [],
    "Min Markups": [],
    "Mean Freq. of Adj.": [],
    "S.D. Freq. of Adj.": [],
    "Max Freq. of Adj.": [],
    "Min Freq. of Adj.": [],
    "Mean Size of Adj.": [],
    "S.D. Size of Adj.": [],
    "Max Size of Adj.": [],
    "Min Size of Adj.": [],
    "Mean Agg. Markup": [],
    "S.D. log C": [],
    "IRs": [],
    "cum_IRs": [],
}
# epsilon_g_pereps = [
#     epsilon_g_list[i * ENV_HORIZON : i * ENV_HORIZON + ENV_HORIZON]
#     for i in range(SIMUL_EPISODES)
# ]
# log_c_filt_pereps = [
#     log_c_filt_list[i * ENV_HORIZON : i * ENV_HORIZON + ENV_HORIZON]
#     for i in range(SIMUL_EPISODES)
# ]
# freq_adj_lowmu_pereps = [
#     freq_adj_lowmu_filt_list[i * ENV_HORIZON : i * ENV_HORIZON + ENV_HORIZON]
#     for i in range(SIMUL_EPISODES)
# ]
# freq_adj_highmu_pereps = [
#     freq_adj_highmu_filt_list[i * ENV_HORIZON : i * ENV_HORIZON + ENV_HORIZON]
#     for i in range(SIMUL_EPISODES)
# ]
# size_adj_lowmu_pereps = [
#     size_adj_lowmu_filt_list[i * ENV_HORIZON : i * ENV_HORIZON + ENV_HORIZON]
#     for i in range(SIMUL_EPISODES)
# ]
# size_adj_highmu_pereps = [
#     size_adj_highmu_filt_list[i * ENV_HORIZON : i * ENV_HORIZON + ENV_HORIZON]
#     for i in range(SIMUL_EPISODES)
# ]
delta_log_c = [j - i for i, j in zip(log_c_filt_list[:-1], log_c_filt_list[1:])]


# print("log_c_filt:", log_c_filt_list, "\n",
#     #"delta_log_c:", delta_log_c,
#     "\n"
print(
    "corr betweeen cons:",
    np.corrcoef(log_c_list, log_c_list_noagg),
)
print(
    "corr betweeen z:",
    np.corrcoef(z_list, z_list_noagg),
)
plt.plot(log_c_filt_list)
plt.title("A. Log C filtered")
# plt.show()
plt.close()

IRs = [0 for t in range(13)]
IRs_freqlow = [0 for t in range(13)]
IRs_freqhigh = [0 for t in range(13)]
IRs_sizelow = [0 for t in range(13)]
IRs_sizehigh = [0 for t in range(13)]
for t in range(0, 13):
    epsilon_g_reg = epsilon_g_list[: -(t + 1)] 
    delta_log_c_reg = delta_log_c[t:] 
    freq_adj_lowmu_reg = freq_adj_lowmu_list[t+1:]
    freq_adj_highmu_reg = freq_adj_highmu_list[t+1:]
    size_adj_lowmu_reg = size_adj_lowmu_list[t+1:] 
    size_adj_highmu_reg = size_adj_highmu_list[t+1:] 

    epsilon_g_reg_filt = [i for i in epsilon_g_reg if i > 0]
    delta_log_c_reg_filt = [
        delta_log_c_reg[i]
        for i in range(len(epsilon_g_reg))
        if epsilon_g_reg[i] > 0
    ]
    freq_adj_lowmu_reg_filt = [
        freq_adj_lowmu_reg[i]
        for i in range(len(epsilon_g_reg))
        if epsilon_g_reg[i] > 0
    ]
    freq_adj_highmu_reg_filt = [
        freq_adj_highmu_reg[i]
        for i in range(len(epsilon_g_reg))
        if epsilon_g_reg[i] > 0
    ]
    size_adj_lowmu_reg_filt = [
        size_adj_lowmu_reg[i]
        for i in range(len(epsilon_g_reg))
        if epsilon_g_reg[i] > 0
    ]
    size_adj_highmu_reg_filt = [
        size_adj_highmu_reg[i]
        for i in range(len(epsilon_g_reg))
        if epsilon_g_reg[i] > 0
    ]
    # epsilon_g_reg_filt = [i for i in epsilon_g_reg if i>0.007]
    # delta_log_c_reg_filt = [delta_log_c_reg[i] for i in range(len(epsilon_g_reg)) if epsilon_g_reg[i]>0.007]
    # freq_adj_lowmu_reg_filt = [freq_adj_lowmu_reg[i] for i in range(len(epsilon_g_reg)) if epsilon_g_reg[i]>0.007]
    # freq_adj_highmu_reg_filt = [freq_adj_highmu_reg[i] for i in range(len(epsilon_g_reg)) if epsilon_g_reg[i]>0.007]
    # size_adj_lowmu_reg_filt = [size_adj_lowmu_reg[i] for i in range(len(epsilon_g_reg)) if epsilon_g_reg[i]>0.007]
    # size_adj_highmu_reg_filt = [size_adj_highmu_reg[i] for i in range(len(epsilon_g_reg)) if epsilon_g_reg[i]>0.007]

    # regressions
    reg_c = linregress(delta_log_c_reg, epsilon_g_reg)
    IRs[t] = reg_c[0] * env.params["sigma_g"] * 100
    reg_freqlow = linregress(freq_adj_lowmu_reg_filt, epsilon_g_reg_filt)
    IRs_freqlow[t] = reg_freqlow[0] * env.params["sigma_g"] * 100
    reg_freqhigh = linregress(freq_adj_highmu_reg_filt, epsilon_g_reg_filt)
    IRs_freqhigh[t] = reg_freqhigh[0] * env.params["sigma_g"] * 100
    reg_sizelow = linregress(size_adj_lowmu_reg_filt, epsilon_g_reg_filt)
    IRs_sizelow[t] = reg_sizelow[0] * env.params["sigma_g"] * 100
    reg_sizehigh = linregress(size_adj_highmu_reg_filt, epsilon_g_reg_filt)
    IRs_sizehigh[t] = reg_sizehigh[0] * env.params["sigma_g"] * 100
cum_IRs = [np.sum(IRs[:t]) for t in range(13)]
cum_IRs_freqlow = [np.sum(IRs_freqlow[:t]) for t in range(13)]
cum_IRs_freqhigh = [np.sum(IRs_freqhigh[:t]) for t in range(13)]
cum_IRs_sizelow = [np.sum(IRs_sizelow[:t]) for t in range(13)]
cum_IRs_sizehigh = [np.sum(IRs_sizehigh[:t]) for t in range(13)]

print(
    "cum_IRs_freqlow:",
    cum_IRs_freqlow[3],
    "\n",
    "cum_IRs_freqhigh:",
    cum_IRs_freqhigh[3],
    "\n",
    "cum_IRs_sizelow:",
    cum_IRs_sizelow[3],
    "\n",
    "cum_IRs_sizehigh:",
    cum_IRs_sizehigh[3],
    "\n",
)

simul_results_dict["Mean Profits"].append(np.mean(profits_list))
simul_results_dict["S.D. Profits"].append(np.std(profits_list))
simul_results_dict["Max Profits"].append(np.max(profits_list))
simul_results_dict["Min Profits"].append(np.min(profits_list))
simul_results_dict["Mean Markups"].append(np.mean(mu_ij_list))
simul_results_dict["S.D. Markups"].append(np.std(mu_ij_list))
simul_results_dict["Max Markups"].append(np.max(mu_ij_list))
simul_results_dict["Min Markups"].append(np.min(mu_ij_list))
simul_results_dict["Mean Freq. of Adj."].append(np.mean(freq_p_adj_list))
simul_results_dict["S.D. Freq. of Adj."].append(np.std(freq_p_adj_list))
simul_results_dict["Max Freq. of Adj."].append(np.max(freq_p_adj_list))
simul_results_dict["Min Freq. of Adj."].append(np.min(freq_p_adj_list))
simul_results_dict["Mean Size of Adj."].append(np.mean(size_adj_list))
simul_results_dict["S.D. Size of Adj."].append(np.std(size_adj_list))
simul_results_dict["Max Size of Adj."].append(np.max(size_adj_list))
simul_results_dict["Min Size of Adj."].append(np.min(size_adj_list))
simul_results_dict["Mean Agg. Markup"].append(np.mean(mu_list))
simul_results_dict["S.D. log C"].append(np.std(log_c_filt_list))
simul_results_dict["IRs"].append(IRs)
simul_results_dict["cum_IRs"].append(cum_IRs)
# simul_results_dict["IRs_freqlow"].append(IRs_freqlow)
# simul_results_dict["IRs_freqhigh"].append(IRs_freqhigh)
# simul_results_dict["IRs_sizelow"].append(IRs_sizelow)
# simul_results_dict["IRs_sizehigh"].append(IRs_sizehigh)

print("Simul_results_dict:", simul_results_dict)
# print(
#     "std_log_c:",
#     simul_results_dict["S.D. log C"],
#     "\n" + "mu_ij:",
#     simul_results_dict["Mean Markups"],
#     "\n" + "freq_p_adj:",
#     simul_results_dict["Mean Freq. of Adj."],
#     "\n" + "size_adj:",
#     simul_results_dict["Mean Size of Adj."],
# )

""" Plot IRs """
x = [i for i in range(13)]
IRs = simul_results_dict["IRs"][-1]
plt.plot(x, IRs)
# learning_plot = learning_plot.get_figure()
plt.ylabel("Delta log C_t * 100")
plt.xlabel("Month t")
plt.ylim([-2.5,15])
# plt.title("A. IRF - Consumption")
# plt.title("MIN")
plt.savefig(OUTPUT_PATH_FIGURES + "IRs_" + exp_names[0] + "_min" + ".png")
plt.show()
plt.close()

cum_IRs = simul_results_dict["cum_IRs"][-1]
plt.plot(x, cum_IRs)
# learning_plot = learning_plot.get_figure()
plt.ylabel("Delta log C_t * 100")
plt.xlabel("Month t")

# plt.title("B. Cumulative IRF - Consumption")
# plt.title("MIN")
plt.savefig(OUTPUT_PATH_FIGURES + "cum_IRs_" + exp_names[0] + "_min" + ".png")
plt.show()
plt.close()

plt.plot(x, IRs_freqlow)
plt.plot(x, IRs_freqhigh)
plt.legend(["Low Markup Firms", "High Markup Firms"])
# learning_plot = learning_plot.get_figure()
plt.ylabel("IRF - Levels * 100")
plt.xlabel("Month t")
# plt.title("IRF - Frquency of Adjustment for High vs Low Markup Firms")
# plt.title("MIN")
plt.savefig(OUTPUT_PATH_FIGURES + "IRs_freq_" + exp_names[0] + "_min" + ".png")
plt.show()
plt.close()

plt.plot(x, IRs_sizelow)
plt.plot(x, IRs_sizehigh)
plt.legend(["Low Markup Firms", "High Markup Firms"])
# learning_plot = learning_plot.get_figure()
plt.ylabel("IRF - Levels * 100")
plt.xlabel("Month t")

# plt.title("IRF - Size of Adjustment for High vs Low Markup Firms")
# plt.title("MIN")
plt.savefig(OUTPUT_PATH_FIGURES + "IRs_size_" + exp_names[0] + "_min" + ".png")
plt.show()
plt.close()